# Python for Genomics 
### by Guilherme Matos Passarini, phD

## Exercise 1: Calculate Recombination Frequency and Estimate Gene Distances
### Write a program that accepts inputs and estimated gene distances based on the frequency of the Recombinant and Non-Recombinant Genotypes.

In [1]:
AB = int(input('Please enter total count of observed AB, non-recombinants: '))
ab = int(input('Please enter total count of observed ab, non-recombinants: '))
aB = int(input('Please enter total count of observed aB, recombinants: '))
Ab = int(input('Please enter total count of observed Ab, recombinants: '))

Please enter total count of observed AB, non-recombinants: 121
Please enter total count of observed ab, non-recombinants: 200
Please enter total count of observed aB, recombinants: 23
Please enter total count of observed Ab, recombinants: 21


In [2]:
freq = (Ab + aB) / (AB + ab + Ab + aB)
print(f'Estimated Recombinant Frequency: {freq:.4f}')

Estimated Recombinant Frequency: 0.1205


In [3]:
distance = (AB + ab + Ab + aB) / (Ab + aB)
print(f'Estimated Gene Distances: {distance:.2f} bases / kB / MB')

Estimated Gene Distances: 8.30 bases / kB / MB


## Exercise 2: Calculate Allele Frequencies
### Write a program that accepts Genotype Counts and returns the Frequencies of the two Alleles.

In [4]:
pp = int(input('Please enter total count of observed pp genotypes: '))
pq = int(input('Please enter total count of observed pq genotypes: '))
qq = int(input('Please enter total count of observed qq genotypes: '))
n = pp+pq+qq

Please enter total count of observed pp genotypes: 13
Please enter total count of observed pq genotypes: 80
Please enter total count of observed qq genotypes: 30


In [5]:
print(f'Total observed genotypes: {n}')

Total observed genotypes: 123


In [6]:
p_freq = ((2*pp)+pq) / (2*n)
print(f'Frequency of Allele P: {p_freq:.4f}')

Frequency of Allele P: 0.4309


In [7]:
q_freq = ((2*qq)+pq) / (2*n)
print(f'Frequency of Allele Q: {q_freq:.4f}')

Frequency of Allele Q: 0.5691


In [8]:
p_freq + q_freq

1.0

## To summarize:

In [9]:
print(f'Frequency of Allele P: {p_freq:.4f}')
print(f'Frequency of Allele Q: {q_freq:.4f}')
print(f'Total: {(p_freq + q_freq):.1f}')

Frequency of Allele P: 0.4309
Frequency of Allele Q: 0.5691
Total: 1.0
